In [1]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency
import matplotlib.pyplot as plt
import io
import os
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

In [2]:
def read_vcf(path):
    with open(path, 'r') as f:
        lines = [l for l in f if not l.startswith('##')]
    return pd.read_csv(
        io.StringIO(''.join(lines)),
        dtype={'#CHROM': str, 'POS': int, 'ID': str, 'REF': str, 'ALT': str,
               'QUAL': str, 'FILTER': str, 'INFO': str},
        sep='\t'
    ).rename(columns={'#CHROM': 'CHROM'})

## hgsvc 9 add 4DN 8 samples 

In [3]:
pangenie_hgsvc_17sample = read_vcf("/data/home/tun53987/Hi-C/pangenie_merged_bi_nosnvs_hgsvc_9add8sample.vcf")

In [4]:
pangenie_hgsvc_17sample.shape

(1129810, 26)

In [5]:
pangenie_hgsvc_17sample[['INFO_ID']] = pangenie_hgsvc_17sample['INFO'].str.split(';', expand=True)[2]
pangenie_hgsvc_17sample['INFO_ID'] = pangenie_hgsvc_17sample['INFO_ID'].str.lstrip('ID=')

In [6]:
pangenie_DEL_hgsvc17sample = pangenie_hgsvc_17sample[pangenie_hgsvc_17sample['INFO_ID'].str.contains('DEL')]
pangenie_INS_hgsvc17sample = pangenie_hgsvc_17sample[pangenie_hgsvc_17sample['INFO_ID'].str.contains('INS')]

In [7]:
pangenie_hgsvc_17sample[['type']] = pangenie_hgsvc_17sample['INFO_ID'].str.split('-', expand=True)[2]

In [8]:
col_list = pangenie_hgsvc_17sample['type'].values.tolist()
pd.Series(col_list).value_counts()

DEL    672432
INS    457378
dtype: int64

#### deletions

In [9]:
pangenie_DEL_hgsvc17sample_new = pangenie_DEL_hgsvc17sample[[
 'CHROM','FILTER', 'INFO_ID', 'FORMAT', 'HG00512','HG00513','HG00514','HG00731','HG00732','HG00733','NA19238','NA19239','NA19240','NA18507','NA18505','NA18508','NA18486','NA19099','NA19141','NA18516','NA18522'
     ]]

In [10]:
for col in pangenie_DEL_hgsvc17sample_new.iloc[:, 4:]:
    pangenie_DEL_hgsvc17sample_new[col] = pangenie_DEL_hgsvc17sample_new[col].str[:3]

/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
for col in pangenie_DEL_hgsvc17sample_new.columns:
    pangenie_DEL_hgsvc17sample_new.loc[pangenie_DEL_hgsvc17sample_new[col].astype(str).str.startswith('0/0'), col] = '0'
    pangenie_DEL_hgsvc17sample_new.loc[pangenie_DEL_hgsvc17sample_new[col].astype(str).str.startswith('.:.'), col] = '0'
    pangenie_DEL_hgsvc17sample_new.loc[pangenie_DEL_hgsvc17sample_new[col].astype(str).str.startswith('0/1'), col] = '1'
    pangenie_DEL_hgsvc17sample_new.loc[pangenie_DEL_hgsvc17sample_new[col].astype(str).str.startswith('1/0'), col] = '1'
    pangenie_DEL_hgsvc17sample_new.loc[pangenie_DEL_hgsvc17sample_new[col].astype(str).str.startswith('1/1'), col] = '1'

/home/tun53987/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/tun53987/.local/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [12]:
pangenie_DEL_hgsvc17sample_new.shape

(672432, 21)

In [13]:
pangenie_DEL_hgsvc17sample_new.iloc[:,4: ] = pangenie_DEL_hgsvc17sample_new.iloc[:,4: ].astype(str).astype(int)
pangenie_DEL_hgsvc17sample_new['Sum']=pangenie_DEL_hgsvc17sample_new.iloc[:,4: ].sum(axis=1)
#pangenie_DEL_hgsvc17sample_new_atleast1 = pangenie_DEL_hgsvc17sample_new[pangenie_DEL_hgsvc17sample_new['Sum'].astype(int) >= 3]
pangenie_DEL_hgsvc17sample_new_atleast1 = pangenie_DEL_hgsvc17sample_new[pangenie_DEL_hgsvc17sample_new['Sum'].astype(int) >= 5]
pangenie_DEL_hgsvc17sample_new_atleast1['START_POS'] = pangenie_DEL_hgsvc17sample_new_atleast1['INFO_ID'].str.split('-', expand=True)[1]
pangenie_DEL_hgsvc17sample_new_atleast1['TYPE'] = pangenie_DEL_hgsvc17sample_new_atleast1['INFO_ID'].str.split('-', expand=True)[2]
pangenie_DEL_hgsvc17sample_new_atleast1['LEN'] = pangenie_DEL_hgsvc17sample_new_atleast1['INFO_ID'].str.split('-', expand=True)[3]

/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [14]:
pangenie_DEL_hgsvc17sample_new_atleast1.shape

(227073, 25)

In [15]:
pangenie_DEL_hgsvc17sample_new_atleast1.iloc[:,[22,24]] = pangenie_DEL_hgsvc17sample_new_atleast1.iloc[:,[22,24]].astype(str).astype(int)
# VCF files in fact are providing a single 1-based position for a variant, end = start + len -1
pangenie_DEL_hgsvc17sample_new_atleast1['END_POS'] = pangenie_DEL_hgsvc17sample_new_atleast1['START_POS'] + pangenie_DEL_hgsvc17sample_new_atleast1['LEN']-1
pangenie_DEL_hgsvc17sample_new_atleast1 = pangenie_DEL_hgsvc17sample_new_atleast1[[
'CHROM', 'START_POS','END_POS', 'TYPE','LEN', 'FILTER', 'INFO_ID', 'FORMAT', 'HG00512','HG00513','HG00514','HG00731','HG00732','HG00733','NA19238','NA19239','NA19240','NA18507','NA18505','NA18508','NA18486','NA19099','NA19141','NA18516','NA18522', 'Sum'
    ]]

/home/tun53987/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
pangenie_DEL_hgsvc17sample_new_atleast1_50 = pangenie_DEL_hgsvc17sample_new_atleast1[pangenie_DEL_hgsvc17sample_new_atleast1['LEN'] >= 50]
pangenie_DEL_hgsvc17sample_new_atleast1_50.shape

(12132, 26)

In [17]:
pangenie_DEL_hgsvc17sample_new_atleast1_50_zero = pangenie_DEL_hgsvc17sample_new_atleast1_50[pangenie_DEL_hgsvc17sample_new_atleast1_50.iloc[:, 8:].eq(0).any(1)]
pangenie_DEL_hgsvc17sample_new_atleast1_50_zero.shape

(10573, 26)

In [18]:
pangenie_DEL_hgsvc17sample_new_atleast1_50_start_end = pangenie_DEL_hgsvc17sample_new_atleast1_50_zero[['CHROM', 'START_POS', 'END_POS', 'INFO_ID']]
pangenie_DEL_hgsvc17sample_new_atleast1_50_start_end.shape

(10573, 4)

#### insertions

In [19]:
pangenie_INS_hgsvc17sample_new = pangenie_INS_hgsvc17sample[[
 'CHROM','FILTER', 'INFO_ID', 'FORMAT', 'HG00512','HG00513','HG00514','HG00731','HG00732','HG00733','NA19238','NA19239','NA19240','NA18507','NA18505','NA18508','NA18486','NA19099','NA19141','NA18516','NA18522'
     ]]

In [20]:
for col in pangenie_INS_hgsvc17sample_new.iloc[:, 4:]:
    pangenie_INS_hgsvc17sample_new[col] = pangenie_INS_hgsvc17sample_new[col].str[:3]

/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [21]:
for col in pangenie_INS_hgsvc17sample_new.columns:
    pangenie_INS_hgsvc17sample_new.loc[pangenie_INS_hgsvc17sample_new[col].astype(str).str.startswith('0/0'), col] = '0'
    pangenie_INS_hgsvc17sample_new.loc[pangenie_INS_hgsvc17sample_new[col].astype(str).str.startswith('.:.'), col] = '0'
    pangenie_INS_hgsvc17sample_new.loc[pangenie_INS_hgsvc17sample_new[col].astype(str).str.startswith('0/1'), col] = '1'
    pangenie_INS_hgsvc17sample_new.loc[pangenie_INS_hgsvc17sample_new[col].astype(str).str.startswith('1/0'), col] = '1'
    pangenie_INS_hgsvc17sample_new.loc[pangenie_INS_hgsvc17sample_new[col].astype(str).str.startswith('1/1'), col] = '1'

/home/tun53987/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/home/tun53987/.local/lib/python3.7/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [22]:
pangenie_INS_hgsvc17sample_new.iloc[:,4: ] = pangenie_INS_hgsvc17sample_new.iloc[:,4: ].astype(str).astype(int)
pangenie_INS_hgsvc17sample_new['Sum']=pangenie_INS_hgsvc17sample_new.iloc[:,4: ].sum(axis=1)
#pangenie_INS_hgsvc17sample_new_atleast1 = pangenie_INS_hgsvc17sample_new[pangenie_INS_hgsvc17sample_new['Sum'].astype(int) >= 3]
pangenie_INS_hgsvc17sample_new_atleast1 = pangenie_INS_hgsvc17sample_new[pangenie_INS_hgsvc17sample_new['Sum'].astype(int) >= 5]
pangenie_INS_hgsvc17sample_new_atleast1['START_POS'] = pangenie_INS_hgsvc17sample_new_atleast1['INFO_ID'].str.split('-', expand=True)[1]
pangenie_INS_hgsvc17sample_new_atleast1['TYPE'] = pangenie_INS_hgsvc17sample_new_atleast1['INFO_ID'].str.split('-', expand=True)[2]
pangenie_INS_hgsvc17sample_new_atleast1['LEN'] = pangenie_INS_hgsvc17sample_new_atleast1['INFO_ID'].str.split('-', expand=True)[3]

/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: htt

In [23]:
pangenie_INS_hgsvc17sample_new_atleast1.iloc[:,[22,24]] = pangenie_INS_hgsvc17sample_new_atleast1.iloc[:,[22,24]].astype(str).astype(int)
pangenie_INS_hgsvc17sample_new_atleast1['END_POS'] = pangenie_INS_hgsvc17sample_new_atleast1['START_POS']
pangenie_INS_hgsvc17sample_new_atleast1_new = pangenie_INS_hgsvc17sample_new_atleast1[[
'CHROM', 'START_POS','END_POS', 'TYPE','LEN', 'FILTER', 'INFO_ID', 'FORMAT', 'HG00512','HG00513','HG00514','HG00731','HG00732','HG00733','NA19238','NA19239','NA19240','NA18507','NA18505','NA18508','NA18486','NA19099','NA19141','NA18516','NA18522', 'Sum'
    ]]

/home/tun53987/.local/lib/python3.7/site-packages/pandas/core/indexing.py:1754: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)
/home/tun53987/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
pangenie_INS_hgsvc17sample_new_atleast1_50 = pangenie_INS_hgsvc17sample_new_atleast1_new[pangenie_INS_hgsvc17sample_new_atleast1_new['LEN'] >= 50]
pangenie_INS_hgsvc17sample_new_atleast1_50.shape

(17056, 26)

In [25]:
pangenie_INS_hgsvc17sample_new_atleast1_50_zero = pangenie_INS_hgsvc17sample_new_atleast1_50[pangenie_INS_hgsvc17sample_new_atleast1_50.iloc[:, 8:].eq(0).any(1)]
pangenie_INS_hgsvc17sample_new_atleast1_50_zero.shape

(14030, 26)

In [26]:
pangenie_INS_hgsvc17sample_new_atleast1_50_start_end = pangenie_INS_hgsvc17sample_new_atleast1_50_zero[['CHROM', 'START_POS', 'END_POS', 'INFO_ID']]
pangenie_INS_hgsvc17sample_new_atleast1_50_start_end.shape

(14030, 4)

## SVs' impact on boundary strength

In [27]:
#all_DEL_merged = pd.read_csv('/home/tun53987/Hi-C/experiment_mergeall_new/mega/aligned/fanc_insulation_SCALE/all_DEL_hgsvc17merged_cutoff_flank_inter30_5kb_boundaries_100kb_only01_atleast3_new10kb.bed', sep='\t', header=None)
all_DEL_merged = pd.read_csv('/home/tun53987/Hi-C/experiment_mergeall_new/mega/aligned/fanc_insulation_SCALE/all_DEL_hgsvc17merged_cutoff_flank_inter30_5kb_boundaries_100kb_only01_atleast5_new10kb.bed', sep='\t', header=None)
all_DEL_merged.columns =['CHROM', 'START_POS', 'END_POS', 'INFO_ID', 'BOUND_CHR', 'FLANK_START', 'FLANK_END', 'BOUND_SCORE']

In [28]:
all_DEL_merged.shape

(10615, 8)

In [29]:
#all_INS_merged = pd.read_csv('/home/tun53987/Hi-C/experiment_mergeall_new/mega/aligned/fanc_insulation_SCALE/all_INS_hgsvc17merged_cutoff_flank_inter30_5kb_boundaries_100kb_only01_atleast3_new10kb.bed', sep='\t', header=None)
all_INS_merged = pd.read_csv('/home/tun53987/Hi-C/experiment_mergeall_new/mega/aligned/fanc_insulation_SCALE/all_INS_hgsvc17merged_cutoff_flank_inter30_5kb_boundaries_100kb_only01_atleast5_new10kb.bed', sep='\t', header=None)
all_INS_merged.columns =['CHROM', 'START_POS', 'END_POS', 'INFO_ID', 'BOUND_CHR', 'FLANK_START', 'FLANK_END', 'BOUND_SCORE']

In [30]:
all_INS_merged.shape

(14067, 8)

In [31]:
all_DEL_boundary_1 = all_DEL_merged.loc[all_DEL_merged.iloc[:, 4] != '.']
all_DEL_boundary_2 = all_DEL_boundary_1.reset_index(drop=True)
all_DEL_boundary_2.shape

(1519, 8)

In [32]:
all_INS_boundary_1 = all_INS_merged.loc[all_INS_merged.iloc[:, 4] != '.']
all_INS_boundary_2 = all_INS_boundary_1.reset_index(drop=True)
all_INS_boundary_2.shape

(2123, 8)

In [33]:
sample_ids = ['HG00512','HG00513','HG00514_1','HG00731','HG00732','HG00733_1','NA19238','NA19239','NA19240_1',
              'GM18507','GM18505','GM18508','GM18486','GM19099','GM19141','GM18516','GM18522']
file_directory = '/home/tun53987/Hi-C'
import glob
for i in sample_ids:
    file_pattern = os.path.join(file_directory, f"*_{i}/aligned/fanc_insulation_SCALE_new/BS_cutoff_inter30_10kb.boundaries_100kb_new10kb.bed")
    sample_file = glob.glob(file_pattern)
    sample_merged_BS = pd.read_csv(sample_file[0], sep='\t', header=None)
    sample_merged_BS =sample_merged_BS[[0, 1, 2, 3, 4, 5, 6, 8]]
    sample_merged_BS.columns =['BOUND_CHR', 'FLANK_START', 'FLANK_END', 'BOUND_SCORE','BOUND_CHR_samp', 'BOUND_START_samp', 'BOUND_END_samp', 'BOUND_SCORE_samp']
    sample_merged_BS_1 = sample_merged_BS.groupby(['BOUND_CHR', 'FLANK_START', 'FLANK_END'], as_index=False)['BOUND_SCORE_samp'].median()
    all_DEL_boundary_2[i] = all_DEL_boundary_2.merge(sample_merged_BS_1, how="left", left_on=["BOUND_CHR", "FLANK_START", "FLANK_END"], right_on=["BOUND_CHR", "FLANK_START", "FLANK_END"])['BOUND_SCORE_samp']
    all_INS_boundary_2[i] = all_INS_boundary_2.merge(sample_merged_BS_1, how="left", left_on=["BOUND_CHR", "FLANK_START", "FLANK_END"], right_on=["BOUND_CHR", "FLANK_START", "FLANK_END"])['BOUND_SCORE_samp']

In [34]:
all_DEL_boundary_2.shape

(1519, 25)

In [35]:
all_INS_boundary_2.shape

(2123, 25)

#### Mann-Whitney U test for genotypes and boundary BS

In [36]:
all_DEL_boundary_2['num_NaNs'] = all_DEL_boundary_2.iloc[:, 8:].astype(float).isna().sum(1)
all_INS_boundary_2['num_NaNs'] = all_INS_boundary_2.iloc[:, 8:].astype(float).isna().sum(1)

In [37]:
all_DEL_boundary_2['num_NaNs'].max()

17

In [38]:
all_INS_boundary_2['num_NaNs'].max()

17

In [39]:
# all_DEL_boundary_2_noNA = all_DEL_boundary_2[all_DEL_boundary_2.num_NaNs <= 12]
# all_INS_boundary_2_noNA = all_INS_boundary_2[all_INS_boundary_2.num_NaNs <= 12]
all_DEL_boundary_2_noNA = all_DEL_boundary_2[all_DEL_boundary_2.num_NaNs < 17]
all_INS_boundary_2_noNA = all_INS_boundary_2[all_INS_boundary_2.num_NaNs < 17]

In [40]:
# Replace NaN Values with Zeros, because those NA boundary scores mean there is no boundary, so we can changed them to zero
all_DEL_boundary_3 = all_DEL_boundary_2_noNA.fillna(0).drop_duplicates()
all_DEL_boundary_3.shape

(1292, 26)

In [41]:
# Replace NaN Values with Zeros
all_INS_boundary_3 = all_INS_boundary_2_noNA.fillna(0).drop_duplicates()
all_INS_boundary_3.shape

(1809, 26)

In [42]:
all_DEL_boundary_3 = all_DEL_boundary_3.reset_index(drop=True)
all_DEL_boundary_3.shape

(1292, 26)

In [43]:
all_INS_boundary_3 = all_INS_boundary_3.reset_index(drop=True)
all_INS_boundary_3.shape

(1809, 26)

In [44]:
all_DEL_boundary_id = list(all_DEL_boundary_3['INFO_ID'])
DEL_list = list(set(all_DEL_boundary_id))
len(DEL_list)

1269

In [45]:
all_INS_boundary_id = list(all_INS_boundary_3['INFO_ID'])
INS_list = list(set(all_INS_boundary_id))
len(INS_list)

1790

In [46]:
INS_genotype = pd.read_csv('/data/home/tun53987/Hi-C/TAD_SV_qtl/pangenie_INS_hgsvc17sample_ALL_INS_atleast5_50_genotype_only01.bed', sep='\t', header=0)
DEL_genotype = pd.read_csv('/data/home/tun53987/Hi-C/TAD_SV_qtl/pangenie_DEL_hgsvc17sample_new_atleast5_50_genotype_only01.bed', sep='\t', header=0)

In [47]:
DEL_genotype_1 = DEL_genotype.iloc[:, :-1]
INS_genotype_1 = INS_genotype.iloc[:, :-1]
DEL_boundary_genotype = DEL_genotype_1[DEL_genotype_1['INFO_ID'].isin(all_DEL_boundary_id)]
INS_boundary_genotype = INS_genotype_1[INS_genotype_1['INFO_ID'].isin(all_INS_boundary_id)]

In [48]:
DEL_boundary_genotype = DEL_boundary_genotype[[
'CHROM', 'START_POS','END_POS', 'TYPE','LEN', 'FILTER', 'INFO_ID', 'FORMAT', 'HG00512','HG00513','HG00514','HG00731','HG00732','HG00733','NA19238','NA19239','NA19240','NA18507','NA18505','NA18508','NA18486','NA19099','NA19141','NA18516','NA18522'
    ]]
INS_boundary_genotype = INS_boundary_genotype[[
'CHROM', 'START_POS','END_POS', 'TYPE','LEN', 'FILTER', 'INFO_ID', 'FORMAT', 'HG00512','HG00513','HG00514','HG00731','HG00732','HG00733','NA19238','NA19239','NA19240','NA18507','NA18505','NA18508','NA18486','NA19099','NA19141','NA18516','NA18522'
    ]]

In [49]:
DEL_boundary_genotype_1 = DEL_boundary_genotype.reset_index(drop=True)
DEL_boundary_genotype_1.shape

(1269, 25)

In [50]:
INS_boundary_genotype_1 = INS_boundary_genotype.reset_index(drop=True)
INS_boundary_genotype_1.shape

(1790, 25)

In [51]:
### test for genotypes 0 and 1 (binary genotypes, only 0/0 and others)
pvalue= []

for i in range(0, len(all_DEL_boundary_3)):
    DEL_boundary_genotype_all = DEL_boundary_genotype_1[DEL_boundary_genotype_1.loc[:,'INFO_ID'] == all_DEL_boundary_3.loc[i,'INFO_ID']].iloc[:,8:]
    genotypes_list = DEL_boundary_genotype_all.iloc[0].values.tolist()
    boundary_score_list = all_DEL_boundary_3.iloc[i,8:25].values.tolist()

    df = pd.DataFrame({'genotypes' : genotypes_list,
                   'boundary_score' : boundary_score_list})
    
    df_new = df.assign(BS=df.groupby('genotypes').cumcount()).pivot('BS','genotypes','boundary_score')
      
    try:
        U, p = mannwhitneyu(x=df_new[0].dropna().tolist(), y=df_new[1].dropna().tolist(), method="exact", alternative = 'two-sided')
        #print (p)
        
    except ValueError:
        p = 'NA'
        #print (p)
    
    pvalue.append(p)
    
all_DEL_boundary_3['pvalue_0_1'] = pvalue

In [52]:
all_DEL_boundary_3.shape

(1292, 27)

In [53]:
### test for genotypes 0 and 1 (binary genotypes, only 0/0 and others)
pvalue= []

for i in range(0, len(all_INS_boundary_3)):
    INS_boundary_genotype_all = INS_boundary_genotype_1[INS_boundary_genotype_1.loc[:,'INFO_ID'] == all_INS_boundary_3.loc[i,'INFO_ID']].iloc[:,8:]
    genotypes_list = INS_boundary_genotype_all.iloc[0].values.tolist()
    boundary_score_list = all_INS_boundary_3.iloc[i,8:25].values.tolist()

    df = pd.DataFrame({'genotypes' : genotypes_list,
                   'boundary_score' : boundary_score_list})
    
    df_new = df.assign(BS=df.groupby('genotypes').cumcount()).pivot('BS','genotypes','boundary_score')
      
    try:
        U, p = mannwhitneyu(x=df_new[0].dropna().tolist(), y=df_new[1].dropna().tolist(), method="exact", alternative = 'two-sided')
        #print (p)
        
    except ValueError:
        p = 'NA'
        #print (p)
    
    pvalue.append(p)
    
all_INS_boundary_3['pvalue_0_1'] = pvalue

In [54]:
all_INS_boundary_3.shape

(1809, 27)

In [55]:
### apply fdr correction
def fdr_correction(group):
    _, corrected_pvalues, _, _ = multipletests(group['pvalue_0_1'], method='fdr_bh')
    group['qvalue'] = corrected_pvalues
    return group

In [56]:
corrected_del = all_DEL_boundary_3.groupby('INFO_ID').apply(fdr_correction)
corrected_del.shape

(1292, 28)

In [57]:
all_DEL_boundary_3_sig = corrected_del[corrected_del['qvalue'] < 0.05]
all_DEL_boundary_3_sig.shape

(35, 28)

In [58]:
corrected_ins = all_INS_boundary_3.groupby('INFO_ID').apply(fdr_correction)
corrected_ins.shape

(1809, 28)

In [59]:
all_INS_boundary_3_sig = corrected_ins[corrected_ins['qvalue'] < 0.05]
all_INS_boundary_3_sig.shape

(57, 28)

In [60]:
len(all_DEL_boundary_3_sig['INFO_ID'].drop_duplicates())

34

In [61]:
len(all_INS_boundary_3_sig)

57

In [62]:
len(all_INS_boundary_3_sig['INFO_ID'].drop_duplicates())

57

#### load significant discovery set

In [63]:
all_DEL_boundary_dis  = pd.read_csv('/home/tun53987/Hi-C/experiment_mergeall_new/mega/aligned/fanc_insulation_SCALE/final_ALL_DEL_merged_BS_flank_26sample_pvalue_only01_fillna_new10kb.bed', sep='\t', header=0)

In [64]:
all_DEL_boundary_dis.shape

(1795, 36)

In [65]:
len(all_DEL_boundary_dis['INFO_ID'].drop_duplicates())

1741

In [66]:
all_DEL_boundary_dis_sig = pd.read_csv('/home/tun53987/Hi-C/experiment_mergeall_new/mega/aligned/fanc_insulation_SCALE/all_DEL_boundary_3_sig_nodup_10kbflank.bed', sep='\t', header=0)

In [67]:
len(all_DEL_boundary_dis_sig['INFO_ID'].drop_duplicates())

66

In [68]:
all_INS_boundary_dis = pd.read_csv('/home/tun53987/Hi-C/experiment_mergeall_new/mega/aligned/fanc_insulation_SCALE/final_ALL_INS_merged_BS_flank_26sample_pvalue_only01_fillna_new10kb.bed', sep='\t', header=0)

In [69]:
all_INS_boundary_dis.shape

(2555, 36)

In [70]:
len(all_INS_boundary_dis['INFO_ID'].drop_duplicates())

2514

In [71]:
all_INS_boundary_dis_sig = pd.read_csv('/home/tun53987/Hi-C/experiment_mergeall_new/mega/aligned/fanc_insulation_SCALE/all_INS_boundary_3_sig_10kbflank.bed', sep='\t', header=0)

In [72]:
len(all_INS_boundary_dis_sig['INFO_ID'].drop_duplicates())

120

In [73]:
sig_DEL_boundary_in_dis = all_DEL_boundary_3_sig[all_DEL_boundary_3_sig['INFO_ID'].isin(all_DEL_boundary_dis_sig['INFO_ID'])]
sig_INS_boundary_in_dis = all_INS_boundary_3_sig[all_INS_boundary_3_sig['INFO_ID'].isin(all_INS_boundary_dis_sig['INFO_ID'])]

In [74]:
len(sig_DEL_boundary_in_dis['INFO_ID'].drop_duplicates())

4

In [75]:
len(sig_INS_boundary_in_dis['INFO_ID'].drop_duplicates())

3

In [76]:
DEL_boundary_in_dis = all_DEL_boundary_3[all_DEL_boundary_3['INFO_ID'].isin(all_DEL_boundary_dis_sig['INFO_ID'])]
INS_boundary_in_dis = all_INS_boundary_3[all_INS_boundary_3['INFO_ID'].isin(all_INS_boundary_dis_sig['INFO_ID'])]

In [77]:
len(DEL_boundary_in_dis['INFO_ID'].drop_duplicates())

40

In [78]:
len(INS_boundary_in_dis['INFO_ID'].drop_duplicates())

81

In [79]:
all_DEL_boundary_sort = corrected_del.sort_values(by="qvalue",ascending=True)
all_DEL_boundary_sort.shape

(1292, 28)

In [80]:
all_INS_boundary_sort = corrected_ins.sort_values(by="qvalue",ascending=True)
all_INS_boundary_sort.shape

(1809, 28)

#### top 25% 

In [81]:
all_DEL_boundary_sort_25 = all_DEL_boundary_sort.head(round(len(all_DEL_boundary_sort)*(1/4)))
all_DEL_boundary_sort_25.shape

(323, 28)

In [82]:
all_INS_boundary_sort_25 = all_INS_boundary_sort.head(round(len(all_INS_boundary_sort)*(1/4)))
all_INS_boundary_sort_25.shape

(452, 28)

In [83]:
sig_DEL_boundary_sort_25 = all_DEL_boundary_sort_25[all_DEL_boundary_sort_25['INFO_ID'].isin(all_DEL_boundary_dis_sig['INFO_ID'])]
sig_INS_boundary_sort_25 = all_INS_boundary_sort_25[all_INS_boundary_sort_25['INFO_ID'].isin(all_INS_boundary_dis_sig['INFO_ID'])]

In [84]:
sig_DEL_boundary_sort_25.shape

(14, 28)

In [85]:
len(sig_DEL_boundary_sort_25['INFO_ID'].drop_duplicates())

14

In [86]:
len(sig_INS_boundary_sort_25['INFO_ID'].drop_duplicates())

23

#### top 50% 

In [87]:
all_DEL_boundary_sort_half = all_DEL_boundary_sort.head(round(len(all_DEL_boundary_sort)*(2/4)))
all_DEL_boundary_sort_half.shape

(646, 28)

In [88]:
all_INS_boundary_sort_half = all_INS_boundary_sort.head(round(len(all_INS_boundary_sort)*(2/4)))
all_INS_boundary_sort_half.shape

(904, 28)

In [89]:
sig_DEL_boundary_sort_half = all_DEL_boundary_sort_half[all_DEL_boundary_sort_half['INFO_ID'].isin(all_DEL_boundary_dis_sig['INFO_ID'])]
sig_INS_boundary_sort_half = all_INS_boundary_sort_half[all_INS_boundary_sort_half['INFO_ID'].isin(all_INS_boundary_dis_sig['INFO_ID'])]

In [90]:
len(sig_DEL_boundary_sort_half['INFO_ID'].drop_duplicates())

26

In [91]:
len(sig_INS_boundary_sort_half['INFO_ID'].drop_duplicates())

49

#### top 75% 

In [92]:
all_DEL_boundary_sort_75 = all_DEL_boundary_sort.head(round(len(all_DEL_boundary_sort)*(3/4)))
all_DEL_boundary_sort_75.shape

(969, 28)

In [93]:
all_INS_boundary_sort_75 = all_INS_boundary_sort.head(round(len(all_INS_boundary_sort)*(3/4)))
all_INS_boundary_sort_75.shape

(1357, 28)

In [94]:
sig_DEL_boundary_sort_75 = all_DEL_boundary_sort_75[all_DEL_boundary_sort_75['INFO_ID'].isin(all_DEL_boundary_dis_sig['INFO_ID'])]
sig_INS_boundary_sort_75 = all_INS_boundary_sort_75[all_INS_boundary_sort_75['INFO_ID'].isin(all_INS_boundary_dis_sig['INFO_ID'])]

In [95]:
len(sig_DEL_boundary_sort_75['INFO_ID'].drop_duplicates())

35

In [96]:
len(sig_INS_boundary_sort_75['INFO_ID'].drop_duplicates())

66